<a href="https://colab.research.google.com/github/ezinneanne/farmeyeml/blob/new_branch/farmeyeimg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting Gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import tensorflow as tf #importing tensorflow for deep learning functionality
from tensorflow.keras import layers, models # Import the 'layers' and 'models' submodules from tensorflow.keras
# 'layers' is used to build different types of neural network layers (e.g., Conv2D, Dense, etc.)
# 'models' provides APIs to create and manage models (Sequential and Functional APIs)

from tensorflow.keras.applications import MobileNetV2 # Import the pre-trained MobileNetV2 model from keras.applications
# MobileNetV2 is a lightweight deep convolutional neural network architecture for mobile and edge devices
# It can be used as a feature extractor or a full model for transfer learning

from sklearn.metrics import classification_report # to make classification report for evaluation
import numpy as np # importing numpy for numerical operations

from tensorflow.keras.models import Sequential  # Importing Sequential model for linear stacking of layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pathlib